In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('bbc-text.csv')

In [3]:
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


In [4]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F

In [5]:
import pandas as pd
from nltk import word_tokenize
import numpy as np

In [11]:
import re, string 
import pandas as pd   
from collections import defaultdict
import spacy
from sklearn.manifold import TSNE
from nltk.corpus import stopwords

In [13]:
import nltk

In [14]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [16]:
STOPWORDS = set(stopwords.words('english'))

In [18]:
pip install gensim

     |████████████████████████████████| 24.7MB 1.0MB/s eta 0:00:01    |█████████▊                      | 7.5MB 870kB/s eta 0:00:20     |█████████████████████████████▎  | 22.6MB 84kB/s eta 0:00:25
     |████████████████████████████████| 71kB 713kB/s eta 0:00:01
     |████████████████████████████████| 133kB 370kB/s eta 0:00:01
     |████████████████████████████████| 5.9MB 243kB/s eta 0:00:01
     |████████████████████████████████| 71kB 98kB/s  eta 0:00:011
  Created wheel for smart-open: filename=smart_open-1.9.0-cp37-none-any.whl size=73088 sha256=66be8cc3f280ddd63c192fdadf535efae9196210d9ba89052026f82a38052952
  Stored in directory: /Users/aiymsergazina/Library/Caches/pip/wheels/ab/10/93/5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open
Note: you may need to restart the kernel to use updated packages.


In [19]:
from gensim import models

In [7]:
pip install spacy

     |████████████████████████████████| 14.2MB 1.0MB/s eta 0:00:01
     |████████████████████████████████| 276kB 3.9MB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 2.2MB/s eta 0:00:01
     |████████████████████████████████| 4.0MB 1.2MB/s eta 0:00:01
     |████████████████████████████████| 215kB 860kB/s eta 0:00:01
     |████████████████████████████████| 61kB 5.0MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [20]:
from gensim.models import Word2Vec
%matplotlib inline

In [21]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    # Remove a sentence if it is only one word long
    if len(text) > 2:
        return ' '.join(word for word in text.split() if word not in STOPWORDS)

df_clean = pd.DataFrame(df.text.apply(lambda x: clean_text(x)))

In [23]:
df_clean.head()

,text
0,tv future hands viewers home theatre systems p...
1,worldcom boss left books alone former worldcom...
2,tigers wary farrell gamble leicester say rushe...
3,yeading face newcastle fa cup premiership side...
4,ocean twelve raids box office ocean twelve cri...


In [22]:
nlp = spacy.load('en', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

df_clean["text_lemmatize"] =  df_clean.apply(lambda x: lemmatizer(x['text']), axis=1)
df_clean['text_lemmatize_clean'] = df_clean['text_lemmatize'].str.replace('-PRON-', '')

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [2]:
corpus = [
    'he is a king',
    'she is a queen',
    'he is a man',
    'she is a woman',
    'warsaw is poland capital',
    'berlin is germany capital',
    'paris is france capital',
]

In [3]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

tokenized_corpus = tokenize_corpus(corpus)
print(tokenized_corpus)

[['he', 'is', 'a', 'king'], ['she', 'is', 'a', 'queen'], ['he', 'is', 'a', 'man'], ['she', 'is', 'a', 'woman'], ['warsaw', 'is', 'poland', 'capital'], ['berlin', 'is', 'germany', 'capital'], ['paris', 'is', 'france', 'capital']]


In [4]:
vocabulary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

In [5]:
word2idx

{'he': 0,
 'is': 1,
 'a': 2,
 'king': 3,
 'she': 4,
 'queen': 5,
 'man': 6,
 'woman': 7,
 'warsaw': 8,
 'poland': 9,
 'capital': 10,
 'berlin': 11,
 'germany': 12,
 'paris': 13,
 'france': 14}

In [8]:
window_size = 2
idx_pairs = []
# for each sentence
for sentence in tokenized_corpus:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

In [9]:
idx_pairs[:10]

array([[0, 1],
       [0, 2],
       [1, 0],
       [1, 2],
       [1, 3],
       [2, 0],
       [2, 1],
       [2, 3],
       [3, 1],
       [3, 2]])

In [10]:
def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

In [12]:
embedding_dims = 5
W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
num_epochs = 1010
learning_rate = 0.001

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = Variable(get_input_layer(data)).float()
        y_true = Variable(torch.from_numpy(np.array([target])).long())

        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)
    
        log_softmax = F.log_softmax(z2, dim=0)

        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        loss_val += loss.data.item()
        loss.backward()
        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        W1.grad.data.zero_()
        W2.grad.data.zero_()
    if epo % 10 == 0:    
        print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')

Loss at epo 0: 4.254825830459595
Loss at epo 10: 3.7999127064432416
Loss at epo 20: 3.498934714283262
Loss at epo 30: 3.2741363286972045
Loss at epo 40: 3.096557263817106
Loss at epo 50: 2.95172506911414
Loss at epo 60: 2.8307217018944875
Loss at epo 70: 2.727667771066938
Loss at epo 80: 2.6386418598038808
Loss at epo 90: 2.5610423122133525
Loss at epo 100: 2.4931364706584387
Loss at epo 110: 2.433704020295824
Loss at epo 120: 2.3817618523325237
Loss at epo 130: 2.3363903965268817
Loss at epo 140: 2.2966657723699297
Loss at epo 150: 2.2616808652877807
Loss at epo 160: 2.230599378687995
Loss at epo 170: 2.202701400859015
Loss at epo 180: 2.177400428908212
Loss at epo 190: 2.1542348980903627
Loss at epo 200: 2.132850138630186
Loss at epo 210: 2.112974510874067
Loss at epo 220: 2.0944005523409164
Loss at epo 230: 2.076967859693936
Loss at epo 240: 2.06055126445634
Loss at epo 250: 2.045051031453269
Loss at epo 260: 2.030386288251196
Loss at epo 270: 2.016490370460919
Loss at epo 280: 2.00